## NIC SA (destagionalizzato)

Dal momento che da esploradati.istat.it non è possibile estrarre una serie storica continuativa del NIC, si prova a fare richiesta per accedere all'API di ISTAT

In [ ]:
# CELLA 0
# Serve per assicurarsi di avere installato le versioni supportate delle librerie necessarie per l'esecuzione del codice

#import sys
#!{sys.executable} -m pip install --upgrade pip setuptools
#!{sys.executable} -m pip install --upgrade "pandasdmx>=1.0.0" "pydantic>=1.10.0,<2.0.0" --force-reinstall


In [2]:
# Script per scaricare dati ISTAT con controllo versioni preliminare

import sys
import importlib # Usato per importare moduli dinamicamente e controllare errori
import logging
import pandas as pd

# Configurazione base del logging (verrà usata anche dallo script ISTAT)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def check_libraries_and_versions():
    """
    Controlla la presenza e le versioni di pydantic e pandasdmx.
    Restituisce "True" se le librerie necessarie sono importabili e le versioni sono stampate,
    "False" altrimenti (specialmente se si verifica l'errore ForwardRef).
    """
    print(f"Versione Python: {sys.version}")
    all_ok = True
    try:
        pydantic_module = importlib.import_module("pydantic")
        print(f"Versione pydantic installata: {pydantic_module.__version__}")
        # Verifico se la versione di pydantic è < 2.0.0 come desiderato
        if not (pydantic_module.__version__.startswith("1.")):
            logger.warning(f"ATTENZIONE: La versione di pydantic ({pydantic_module.__version__}) non è < 2.0.0. Potrebbero esserci problemi.")
            # Non imposto all_ok = False qui, ma è un avvertimento importante.
    except ImportError:
        print("LIBRERIA CRITICA MANCANTE: pydantic non è installato.")
        all_ok = False
    except Exception as e:
        print(f"Errore durante l'import o il controllo versione di pydantic: {e}")
        all_ok = False

    if not all_ok: return False # Se pydantic ha problemi, inutile continuare

    try:
        # pandasdmx viene importato qui per testare specificamente l'errore ForwardRef
        # Se questo import fallisce, la funzione restituirà False
        global pandasdmx_Request # Se l'import ha successo, rendiamo Request accessibile globalmente
        from pandasdmx import Request as pdmx_Req
        pandasdmx_Request = pdmx_Req # Assegnazione per uso successivo

        pandasdmx_module = importlib.import_module("pandasdmx")
        print(f"Versione pandasdmx installata: {pandasdmx_module.__version__}")

    except ImportError:
        print("LIBRERIA CRITICA MANCANTE: pandasdmx non è installato.")
        all_ok = False
    except TypeError as te:
        # Questo è il passaggio cruciale per catturare l'errore ForwardRef
        print(f"ERRORE DI TIPO DURANTE L'IMPORT DI PANDASDMX: {te}")
        print("Questo è probabilmente l'errore 'ForwardRef._evaluate() missing ... recursive_guard'.")
        print("Sono stati eseguiti correttamente i comandi pip per installare pydantic < 2.0.0 e RIAVVIATO IL KERNEL???")
        import traceback
        print(traceback.format_exc())
        all_ok = False
    except Exception as e:
        print(f"Errore durante l'import o il controllo versione di pandasdmx: {e}")
        all_ok = False
        
    return all_ok

def run_istat_script():
    """
    Contiene la logica principale per scaricare e processare i dati ISTAT.
    Viene eseguito solo se check_libraries_and_versions() restituisce True.
    """
    logger.info("Avvio dello script per scaricare i dati ISTAT...")
    
    # pandasdmx.Request è stato reso disponibile come pandasdmx_Request
    # se l'import in check_libraries_and_versions è riuscito.
    global pandasdmx_Request

    try:
        # Inizializzo il client (cioè questo script) per ISTAT
        logger.info("Inizializzazione del client ISTAT...")
        istat = pandasdmx_Request('ISTAT') # Uso la Request importata

        # Verifica della disponibilità del dataset
        logger.info("Verifico la disponibilità del dataset...")
        try:
            available_dataflows = istat.dataflow()
            if 'SDDS_PLUS_CPI_DF' not in available_dataflows.dataflow:
                logger.error("Dataset SDDS_PLUS_CPI_DF non trovato.")
                alternative_datasets = [
                    key for key in available_dataflows.dataflow.keys()
                    if isinstance(key, str) and ('CPI' in key.upper() or 'PRICE' in key.upper())
                ]
                if alternative_datasets:
                    logger.info(f"Dataset alternativi disponibili: {alternative_datasets}")
                else:
                    logger.info("Nessun dataset alternativo con 'CPI' o 'PRICE' trovato.")
                sys.exit(1)
        except AttributeError as ae:
            logger.error(f"Errore nella struttura dell'oggetto dataflow restituito da istat.dataflow(): {ae}.")
            logger.info(f"Tipo di oggetto restituito: {type(available_dataflows)}. Contenuto (parziale): {str(available_dataflows)[:500]}")
            sys.exit(1)
        except Exception as e:
            logger.error(f"Errore durante la verifica dei dataset: {e}")
            sys.exit(1)

        # Tento due approcci: prima con destagionalizzazione, poi senza se necessario
        adjustments_to_try = ['SA', None]
        data = None
        success = False
        successful_adjustment = None

        for adjustment_value in adjustments_to_try:
            try:
                key_params = {
                    'FREQ': 'M', 'REF_AREA': 'IT', 'MEASURE': 'I',
                    'PRICE': 'CP00', 'UNIT_MEASURE': 'INX',
                }
                if adjustment_value:
                    key_params['ADJUSTMENT'] = adjustment_value
                
                logger.info(f"Richiesta dati con parametri: {key_params}")
                resp = istat.data(
                    resource_id='SDDS_PLUS_CPI_DF',
                    key=key_params,
                    params={'startPeriod': '2004-01', 'endPeriod': '2024-12'}
                )
                
                # Verifico se ci sono serie di dati nella risposta
                # Alcune versioni/implementazioni di SDMX potrebbero non avere resp.series,
                # o potrebbe essere vuoto se non ci sono dati.
                # to_pandas() dovrebbe comunque gestire la situazione.
                if not hasattr(resp, 'series') or not list(resp.series):
                    # Tento comunque di convertire in pandas, potrebbe esserci un dataset vuoto
                    logger.warning(f"La risposta non sembra contenere serie di dati esplicite con adjustment={adjustment_value}. Tentativo di conversione...")

                data_df = resp.to_pandas() # Converto in pandas DataFrame/Series

                if data_df.empty:
                    logger.warning(f"Dataframe vuoto restituito da to_pandas() con adjustment={adjustment_value}")
                    continue
                    
                logger.info(f"Dati ottenuti con successo con adjustment={adjustment_value}")
                data = data_df
                success = True
                successful_adjustment = adjustment_value
                break # Esce dal loop al primo successo
            except Exception as e_loop:
                logger.warning(f"Tentativo di recupero dati fallito con adjustment={adjustment_value}: {e_loop}")
                # import traceback ---> # Uncomment per debug dettagliato del loop
                # logger.debug(traceback.format_exc())
                continue
        
        if not success:
            logger.error("Impossibile ottenere i dati con i parametri specificati dopo tutti i tentativi.")
            sys.exit(1)

        # Log della struttura del dataframe prima della manipolazione
        logger.info(f"Tipo di dati restituiti da to_pandas(): {type(data)}")
        logger.info(f"Prime righe del dataframe originale:\n{data.head()}")
        if isinstance(data, pd.DataFrame):
            logger.info(f"Indici del dataframe: {data.index.names}")
            logger.info(f"Colonne del dataframe originale: {data.columns.tolist()}")
        elif isinstance(data, pd.Series):
            logger.info(f"Indice della serie: {data.index.name if data.index.name else 'N/A'}")
            logger.info(f"Nome della serie: {data.name if data.name else 'N/A'}")

        # Reset index con gestione di possibili strutture diverse
        df = None
        if isinstance(data, pd.Series):
            df = data.reset_index()
            # Se era una serie, la colonna dei valori potrebbe chiamarsi 0 o col nome della serie
            if df.columns[-1] == 0 or df.columns[-1] == data.name:
                 df.rename(columns={df.columns[-1]: 'OBS_VALUE'}, inplace=True)
        elif isinstance(data, pd.DataFrame):
            df = data.reset_index()
        else:
            logger.error(f"Il tipo di dati {type(data)} non è un DataFrame o Series pandas supportato per l'elaborazione.")
            sys.exit(1)

        logger.info(f"Colonne dopo reset_index: {df.columns.tolist()}")

        # Identifico la colonna temporale e quella dei valori (vado a tentativi su nomi standard)
        time_col, value_col = None, None
        time_candidates = ['TIME_PERIOD', 'time_period', 'TIME', 'Time', 'time', 'Date', 'date', 'Period', 'period']
        value_candidates = ['OBS_VALUE', 'obs_value', 'Value', 'value']

        for tc in time_candidates:
            if tc in df.columns: time_col = tc; break
        for vc in value_candidates:
            if vc in df.columns: value_col = vc; break
        
        # Fallback euristico se i nomi standard non sono trovati
        if not time_col or not value_col:
            logger.warning("Nomi colonna standard (TIME_PERIOD/OBS_VALUE) non trovati. Tentativo con euristica.")
            temp_cols = [col for col in df.columns if col not in ['FREQ', 'REF_AREA', 'MEASURE', 'PRICE', 'UNIT_MEASURE', 'ADJUSTMENT']]
            
            if not value_col: # Identifico colonna valore
                for col_candidate in reversed(temp_cols): # Parto dal fondo, spesso il valore è l'ultima colonna non-dimensione
                    if df[col_candidate].dtype in ('float64', 'int64', 'float32', 'int32', 'float', 'int'):
                        value_col = col_candidate
                        break
                if not value_col and temp_cols: value_col = temp_cols[-1] # Ultima risorsa

            if not time_col: # Identifico colonna tempo
                for col_candidate in temp_cols:
                    if col_candidate == value_col: continue
                    # Controllo se la colonna contiene periodi o date
                    if df[col_candidate].dtype == 'object' and \
                       df[col_candidate].dropna().astype(str).str.match(r'^\d{4}(?:-\d{2})?(?:[QMT]\d{1,2})?$', na=False).any():
                        time_col = col_candidate
                        break
                    # Potrebbe essere già un oggetto datetime/period se pandasdmx lo ha parsato
                    elif pd.api.types.is_datetime64_any_dtype(df[col_candidate]) or \
                         isinstance(df[col_candidate].dtype, pd.PeriodDtype):
                        time_col = col_candidate
                        break
        
        if time_col and value_col:
            logger.info(f"Colonna temporale identificata: '{time_col}'")
            logger.info(f"Colonna valore identificata: '{value_col}'")

            df_final = df[[time_col, value_col]].copy() # Uso .copy() per evitare SettingWithCopyWarning
            df_final.columns = ['Time', 'Value']

            try:
                # Provo a convertire 'Time' in Periodo mensile per un ordinamento robusto
                # Se è già un Periodo o Datetime, la conversione potrebbe essere diretta o non necessaria
                if not isinstance(df_final['Time'].dtype, pd.PeriodDtype):
                    df_final['Time'] = pd.to_datetime(df_final['Time']).dt.to_period('M')
                logger.info("Colonna 'Time' convertita/verificata come pd.PeriodIndex (mensile).")
            except Exception as conv_err:
                logger.warning(f"Impossibile convertire la colonna 'Time' ('{time_col}') in Periodo mensile: {conv_err}. L'ordinamento si baserà sul tipo esistente.")

            df_final.sort_values('Time', inplace=True)

            adjustment_suffix = 'RAW'
            if successful_adjustment == 'SA':
                adjustment_suffix = 'SA'
            
            output_file = f"CPI_NIC_{adjustment_suffix}_Italy_2004_2024.csv"
            df_final.to_csv(output_file, index=False)
            logger.info(f"Dati salvati nel file: {output_file}")

            print("\nPrime righe del dataset estratto:")
            print(df_final.head())
            print("\nStatistiche descrittive:")
            print(df_final['Value'].describe())
            
            missing = df_final['Value'].isna().sum()
            if missing > 0:
                print(f"\nValori mancanti nella colonna 'Value': {missing}")
            else:
                print("\nNessun valore mancante nella colonna 'Value'.")
        else:
            logger.error("Impossibile identificare automaticamente le colonne temporali e/o di valore nel dataframe.")
            print("\nStruttura del dataframe dopo il reset_index():")
            print(df.head())
            print("\nColonne disponibili:", df.columns.tolist())
            if time_col: print(f"Colonna tempo tentata: {time_col}")
            else: print("Colonna tempo NON identificata.")
            if value_col: print(f"Colonna valore tentata: {value_col}")
            else: print("Colonna valore NON identificata.")

    except Exception as e_main:
        logger.error(f"ERRORE CRITICO NELL'ESECUZIONE DELLO SCRIPT ISTAT: {e_main}")
        import traceback
        logger.error(traceback.format_exc()) # Stampo il traceback completo per l'errore principale
        if 'data' in locals() and data is not None: # Verifico se 'data' esiste ed è stato popolato
            print("\nStruttura del dataframe originale (se disponibile) che potrebbe aver causato l'errore:")
            print(data.head())
        sys.exit(1) # Esce in caso di errore nello script principale

# Blocco di esecuzione principale
if __name__ == "__main__":
    # Fase 1: Controllo delle librerie e delle versioni
    logger.info("FASE 1: Controllo delle librerie di base (pydantic, pandasdmx)...")
    if check_libraries_and_versions():
        logger.info("Controllo librerie superato.\n")
        # Fase 2: Esecuzione dello script ISTAT
        logger.info("FASE 2: Avvio estrazione dati ISTAT.")
        run_istat_script()
    else:
        logger.error("Script ISTAT non eseguito a causa di problemi con le librerie di base (pydantic/pandasdmx).")
        logger.error("Rivedi i messaggi di errore sopra. Assicurati di aver eseguito i comandi pip per installare/aggiornare le librerie e di aver RIAVVIATO IL KERNEL del notebook.")
        sys.exit(1) # Esce se il controllo delle librerie fallisce


2025-05-15 00:32:42,676 - INFO - FASE 1: Controllo delle librerie di base (pydantic, pandasdmx)...
2025-05-15 00:32:42,682 - INFO - Controllo librerie superato.

2025-05-15 00:32:42,683 - INFO - FASE 2: Avvio estrazione dati ISTAT.
2025-05-15 00:32:42,684 - INFO - Avvio dello script per scaricare i dati ISTAT...
2025-05-15 00:32:42,688 - INFO - Inizializzazione del client ISTAT...
2025-05-15 00:32:42,704 - INFO - Verifico la disponibilità del dataset...


Versione Python: 3.13.3 (main, Apr  8 2025, 13:54:08) [Clang 16.0.0 (clang-1600.0.26.6)]
Versione pydantic installata: 1.10.22
Versione pandasdmx installata: 1.4.1


2025-05-15 00:32:43,013 - ERROR - Errore durante la verifica dei dataset: 500 Server Error: Internal Server Error for url: https://sdmx.istat.it/SDMXWS/rest/dataflow/all/latest


SystemExit: 1

/Users/tommaso/my_python_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Metodologia

**Verifica della Connettività e Disponibilità del Dataflow**<br>
Come primo passo, si stabilisce una connessione all'endpoint SDMX dell'ISTAT utilizzando la libreria pandasdmx. Si procede al recupero dell'elenco completo dei dataflow disponibili per accertare la funzionalità generale del servizio e per confermare la presenza del dataflow di interesse, identificato come SDDS_PLUS_CPI_DF.

**Recupero e Analisi della Data Structure Definition (DSD)**<br>
Una volta confermata la disponibilità del dataflow, si tenta di recuperare la sua DSD. La DSD è un artefatto SDMX cruciale che descrive la struttura dei dati, includendo l'elenco completo delle dimensioni, i loro identificativi, l'ordine previsto e le codelist associate a ciascuna dimensione.<br>
L'analisi della DSD è fondamentale per comprendere come costruire correttamente la chiave di interrogazione (key) per la richiesta dati, specialmente alla luce delle modifiche segnalate da ISTAT. Si presterà attenzione all'ordine e agli ID esatti delle dimensioni come FREQ, REF_AREA, MEASURE, PRICE, UNIT_MEASURE, e ADJUSTMENT.

**Costruzione della Query e Recupero dei Dati**<br>
Sulla base delle informazioni estratte dalla DSD (in particolare l'ordine e gli ID delle dimensioni), si formula la key per la query dati. Si effettua quindi la richiesta dati al dataflow SDDS_PLUS_CPI_DF specificando i parametri per le dimensioni desiderate ('FREQ': 'M', 'REF_AREA': 'IT', 'MEASURE': 'I', 'PRICE': 'CP00', 'UNIT_MEASURE': 'INX'.) e il periodo temporale di interesse (startPeriod: '2004-01', endPeriod: '2024-12').<br>
Si considera la possibilità di richiedere sia dati destagionalizzati (ADJUSTMENT='SA') sia dati grezzi (omettendo il parametro ADJUSTMENT o specificandone il valore appropriato se indicato dalla DSD per i dati non destagionalizzati).

**Elaborazione e Salvataggio dei Dati**<br>
I dati restituiti dal servizio SDMX vengono convertiti in un formato tabellare pandas.DataFrame. Il DataFrame viene processato per selezionare le colonne rilevanti (il periodo temporale e il valore dell'indice), rinominarle per chiarezza e ordinarle cronologicamente. Infine, i dati elaborati vengono salvati in un file CSV per successive analisi.

**Gestione delle Problematiche Potenziali**<br>
- *Errori del Server (es. HTTP 500)*. Se si verificano errori del server durante una qualsiasi fase, si considera la possibilità di problemi temporanei sul server ISTAT o di una non conformità della richiesta inviata rispetto alle aspettative del nuovo servizio SDMX-RI.<br>
- *Mancato Recupero della DSD*. Se il recupero della DSD fallisce, ma l'elenco dei dataflow è accessibile, ciò potrebbe indicare un problema specifico con la DSD del dataflow SDDS_PLUS_CPI_DF sull'endpoint ISTAT. In tale scenario, la costruzione accurata della query diventa più complessa e dipendente da documentazione esterna o tentativi basati sulle convenzioni note.<br>
- *Dati Mancanti o Inattesi*. Si verifica che i dati restituiti siano coerenti con le aspettative.


**Errore rilevato**<br>
L'output 500 Server Error: Internal Server Error for url: https://sdmx.istat.it/SDMXWS/rest/dataflow/all/latest attesta che il servizio SDMX dell'ISTAT non è funzionante o è instabile. Nonostante si sia provato ad accedervi più volte in giorni diversi, l'output non cambia.

*Monitorare lo Stato del Servizio ISTAT*: Si può provare ad accedere periodicamente a https://sdmx.istat.it/SDMXWS/rest/dataflow/all/latest dal browser, oppure fare un check tramite uno script di controllo apposito, denominato ‘check_istat_sdmx_status.py’.

Tuttavia, dopo aver eseguito più volte lo script ‘check_istat_sdmx_status.py’ dal terminale interno a VS Code, l'output restituito non cambia: ‘ERROR - Il servizio ISTAT SDMX NON sembra essere operativo al momento’. Si deve dunque procedere con l'estrazione manuale delle serie storiche.